In [1]:
import pandas as pd
import numpy as np
import sys
from numpy import NaN
from scipy import stats
import statistics
import matplotlib.pyplot as plt
from sklearn.svm import SVC
#from sklearn.cluster import KMeans
from scipy import signal
from scipy import linalg
from sklearn.decomposition import FastICA, PCA
#from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

#%matplotlib inline  
#%matplotlib notebook
#%pylab

In [2]:
# uncomment this if you want to see all the dataframe
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

In [3]:
RPPA_total = pd.read_excel("Weiqing_2010 data+2011 May pMEK_good pMEK_ed.xlsx",sheet_name='Weiqing_Normalized linear',index_col=1)
# We delete correction factor column CF.1
RPPA_total = RPPA_total.drop('CF.1',axis=1)
RPPA_total = RPPA_total.drop('Sample Order',axis=1)
# columns with "not correct" must be deleted as well
wrong_cols=RPPA_total.filter(regex='not correct').columns
print(wrong_cols)
RPPA_total = RPPA_total.drop(wrong_cols,axis=1)

Index(['MEK1_2_pS217_GBL9007261.txt (not correct)', 'MEK1_2_pS217_GBL9007123.txt(not correct)'], dtype='object')


In [4]:
# reading legend
RPPA_legend = pd.read_excel('slide_plates_sample description.xlsx',sheet_name="WQ's sample description",index_col=1)
RPPA_legend

,Sample Order (in RPPA data from Yiling),Sample Description (drug abbre. | dose or time-point),series / replications on one slide,experimental sets,Notes,Sample Description ( in 11 plates layout),Sample description (drug name & target)
Sample Name (in RPPA data from Yiling),,,,,,,
wq|361,1000.1.1,"AKTi|0.59, 901|0.44",F2,4 drug pert,4/20/10 samples,AKTi + 901 - dose 1,AKTi + 901 - dose 1
wq|362,1001.1.1,"AKTi|1.78, 901|0.88",F2,4 drug pert,4/20/10 samples,AKTi + 901 - dose 2,AKTi + 901 - dose 2
wq|363,1002.1.1,"AKTi|5.33, 901|1.75",F2,4 drug pert,4/20/10 samples,AKTi + 901 - dose 3,AKTi + 901 - dose 3
wq|364,1003.1.1,"Tm|0.15, 901|0.44",F2,4 drug pert,4/20/10 samples,Tm + 901 - dose 1,Tm + 901 - dose 1
wq|365,1004.1.1,"Tm|0.22, 901|0.88",F2,4 drug pert,4/20/10 samples,Tm + 901 - dose 2,Tm + 901 - dose 2
...,...,...,...,...,...,...,...
wq|356,984.1.1,"Tm|0.22, ZS|0.63",F2,4 drug pert,4/20/10 samples,Tm + ZS - dose 2,Tm + ZS - dose 2
wq|357,996.1.1,"Tm|0.33, ZS|1.25",F2,4 drug pert,4/20/10 samples,Tm + ZS - dose 3,Tm + ZS - dose 3
wq|358,997.1.1,ZS|0.63,F2,4 drug pert,4/20/10 samples,ZS - dose 1,ZS - dose 1


In [5]:
# creating groups based on legend
# controls and the rest
controls = RPPA_legend.loc[RPPA_legend['Sample Description (drug abbre. | dose or time-point)'].str.contains('ctrl')]
noncontrols = RPPA_legend.loc[list(set(RPPA_legend.index).difference(set(controls.index)))]

# 12 drugs pert set
drugs12pert = RPPA_legend.loc[RPPA_legend['experimental sets'] == '12 drugs pert']
drugs12pert_ctrl = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains('ctrl')]
drugs12pert_nonctrl = drugs12pert.loc[list(set(drugs12pert.index).difference(set(controls.index)))]

drugs12pert_AKTi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^AK\|\d+\.*\d*$',regex=True)]
drugs12pert_RAFi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^PLX\|\d+\.*\d*$',regex=True)]
drugs12pert_mTORi = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^Tm\|\d+\.*\d*$',regex=True)]
drugs12pert_MEKi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^901\|\d+\.*\d*$',regex=True)]
drugs12pert_CDKi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^RY\|\d+\.*\d*$',regex=True)]
drugs12pert_JAKi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^P6\|\d+\.*\d*$',regex=True)]
drugs12pert_STATi = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^ST\|\d+\.*\d*$',regex=True)]
drugs12pert_MDM2i = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^NT\|\d+\.*\d*$',regex=True)]
drugs12pert_SRCi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^SR\|\d+\.*\d*$',regex=True)]
drugs12pert_PKCi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^RO\|\d+\.*\d*$',regex=True)]
drugs12pert_HDACi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^HN\|\d+\.*\d*$',regex=True)]
drugs12pert_PI3Ki  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^ZS\|\d+\.*\d*$',regex=True)]

drugs12pert_AKTi_MEKi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^AK\|\d+\.*\d*,901\|\d+\.*\d*$',regex=True)]
drugs12pert_PI3Ki_MEKi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^901\|\d+\.*\d*,ZS\|\d+\.*\d*$',regex=True)]
drugs12pert_mTORi_MEKi  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^901\|\d+\.*\d*,Tm\|\d+\.*\d*$',regex=True)]

# PLX dose response
PLX_DR = RPPA_legend.loc[RPPA_legend['experimental sets'] == 'PLX4720 dose series']
PLX_DR_ctrl = PLX_DR.loc[PLX_DR['Sample Description (drug abbre. | dose or time-point)'].str.contains('ctrl')]
PLX_DR_RAFi  = PLX_DR.loc[PLX_DR['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^PLX\|\d+\.*\d*$',regex=True)]

# 4 drugs pert set
drugs4pert = RPPA_legend.loc[RPPA_legend['experimental sets'] == '4 drug pert']
drugs4pert_ctrl = drugs4pert.loc[drugs4pert['Sample Description (drug abbre. | dose or time-point)'].str.contains('ctrl')]
drugs4pert_nonctrl = drugs4pert.loc[list(set(drugs4pert.index).difference(set(controls.index)))]

drugs4pert_AKTi  = drugs4pert.loc[drugs4pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^AKTi\|\d+\.*\d*$',regex=True)]
drugs4pert_mTORi = drugs4pert.loc[drugs4pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^Tm\|\d+\.*\d*$',regex=True)]
drugs4pert_PI3Ki  = drugs4pert.loc[drugs4pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^ ZS\|\d+\.*\d*$',regex=True)]
drugs4pert_MEKi  = drugs4pert.loc[drugs4pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^901\|\d+\.*\d*$',regex=True)]

# EGF stimulation
EGF_TC = RPPA_legend.loc[RPPA_legend['experimental sets'] == 'EGF stimulation']
EGF_TC_ctrl = EGF_TC.loc[EGF_TC['Sample Description (drug abbre. | dose or time-point)'].str.contains('ctrl')]
EGF_TC_LG  = EGF_TC.loc[EGF_TC['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='EGF',regex=True)]



In [6]:
# picking doses for STV
ptb_1 = "AK"
ptb_2 = "ZS"
ptb_3 = 'Tm'

drugs12pert_pert_1  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^' + ptb_1 + '\|\d+\.*\d*$',regex=True)]
dose_list_1 = list(set(drugs12pert_pert_1['Sample Description (drug abbre. | dose or time-point)']))
print(dose_list_1)
drugs12pert_pert_2  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^' + ptb_2 + '\|\d+\.*\d*$',regex=True)]
dose_list_2 = list(set(drugs12pert_pert_2['Sample Description (drug abbre. | dose or time-point)']))
print(dose_list_2)
drugs12pert_pert_3  = drugs12pert.loc[drugs12pert['Sample Description (drug abbre. | dose or time-point)'].str.contains(pat='^' + ptb_3 + '\|\d+\.*\d*$',regex=True)]
dose_list_3 = list(set(drugs12pert_pert_3['Sample Description (drug abbre. | dose or time-point)']))
print(dose_list_3)

# preparing training dataset
#perts_STV = ['AK|10','ZS|1.2','Tm|0.6']
drugs12pert_pert_1high = drugs12pert_pert_1[drugs12pert_pert_1['Sample Description (drug abbre. | dose or time-point)'] == 'AK|10'].index
drugs12pert_pert_2high = drugs12pert_pert_2[drugs12pert_pert_2['Sample Description (drug abbre. | dose or time-point)'] == 'ZS|1.2'].index
drugs12pert_pert_3high = drugs12pert_pert_3[drugs12pert_pert_3['Sample Description (drug abbre. | dose or time-point)'] == 'Tm|0.6'].index
drugs12_pert_index = list(drugs12pert_pert_1high) + list(drugs12pert_pert_2high) + list(drugs12pert_pert_3high)
drugs12_pert_index

['AK|10', 'AK|5']
['ZS|0.6', 'ZS|1.2']
['Tm|0.6', 'Tm|0.3']


['wq|36',
 'wq|172',
 'wq|225',
 'wq|37',
 'wq|176',
 'wq|226',
 'wq|54',
 'wq|184',
 'wq|264']

In [7]:
# building STV

drug12_ctrl = RPPA_total.loc[drugs12pert_ctrl.index]

drug12_AKTi_MEKi = RPPA_total.loc[drugs12pert_AKTi_MEKi.index]
drug12_PI3Ki_MEKi = RPPA_total.loc[drugs12pert_PI3Ki_MEKi.index]
drug12_mTORi_MEKi = RPPA_total.loc[drugs12pert_mTORi_MEKi.index]

drug12_dead = pd.concat([drug12_AKTi_MEKi,drug12_PI3Ki_MEKi,drug12_mTORi_MEKi],axis=0)
ctrl_cnt = drug12_ctrl.mean(axis=0)
dead_cnt = drug12_dead.mean(axis=0)

drug12_trainset = pd.concat([drug12_ctrl,drug12_AKTi_MEKi,drug12_PI3Ki_MEKi,drug12_mTORi_MEKi],axis=0)
cls = list(np.full(len(drug12_ctrl.index),0)) + list(np.full(len(drug12_AKTi_MEKi.index),1)) + list(np.full(len(drug12_PI3Ki_MEKi.index),1)) + list(np.full(len(drug12_mTORi_MEKi.index),1))

#drug12_train = RPPA_total.loc[drugs12_pert_index]

#drug12_trainset = pd.concat([drug12_ctrl,drug12_train],axis=0)
#cls = list(np.full(len(drug12_ctrl.index),0)) + list(np.full(len(drug12_train.index),1))

svc = SVC(kernel='linear')
svc.fit(drug12_trainset,cls)
b_coeff = svc.intercept_[0]/linalg.norm(svc.coef_[0])
norm_vec = - svc.coef_[0]/linalg.norm(svc.coef_[0])
s_vec = (ctrl_cnt - dead_cnt)/linalg.norm(ctrl_cnt - dead_cnt)

norm_vec_df = pd.DataFrame(s_vec,index=RPPA_total.columns,columns=['s_vec'])
norm_vec_df.insert(0,'norm_vec',norm_vec)
# plot STV ranking
norm_vec_df.to_csv('Sander_STV.csv')
norm_vec_df.sort_values(by='s_vec')

# minus is higher in VEM-sensitive state
# plus is higher in VEM-resistant state

,norm_vec,s_vec
SRC_GBL9007059.txt,-0.140135,-0.157848
SRC_GBL9007055.txt,-0.159722,-0.129159
Caveolin_V_GBL9007006.txt,-0.119310,-0.109400
Calveolin.V._GBL9007280.txt,-0.110349,-0.099560
b.Catenin_V_GBL9007000.txt,-0.155847,-0.098230
...,...,...
S6_pS240.V._GBL9007070.txt,0.182366,0.164854
S6_pS235_V_GBL9007270.txt,0.288817,0.258976
CyclinB1.V._GBL9007281.txt,0.229616,0.279245
CyclinB1_V_GBL9007008.txt,0.225351,0.309660


In [8]:
angle = (np.arccos(np.dot(norm_vec,s_vec))/np.pi)*180
angle

18.371021712383346

In [9]:
norm_vec_df.filter(regex="Myc",axis=0)
# Myc has positive contribution to STV, OK

,norm_vec,s_vec
c.Myc_GBL9006959.txt,0.002595,0.002581
c.Myc_pT58_GBL9007257.txt,-0.001495,-0.000713
c.Myc_pT58_GBL9007121.txt,-0.000662,-0.000923
c.Myc_pT58_GBL9007179.txt,-0.000862,-0.000717


In [10]:
# core network: MEK/ERK (ERK), PI3K/AKT (AKT), mTOR/S6K (TOR), SRC, CDK, PKC, IRS
# 6 perturbed, 2 unperturbed, 8 in total
# assigining outputs to different modules

ERK_module_names = ['EGFR_pY1173_C_GBL9007098.txt','EGFR_pY992_V_GBL9007099.txt','EGFR_V_GBL9007097.txt','EGFR_pY1173_C_GBL9007284.txt','EGFR_pY992_V_GBL9007285.txt','EGFR.V._GBL9006975.txt','cRAF_V_GBL9007004.txt','b.RAF_mouse_GBL9007358.txt','b.RAF_pS445_GBL9007254.txt','b.RAF_pS445_GBL9007069.txt','MEK1-2-pS217-221_GBL9006861.txt (correct)','MEK1_V_GBL9007480.txt','MEK1.V._GBL9006980.txt','MAPK_pT202_GBL9007479.txt','MAPK_pT202_GBL9007114.txt','MAPK_pT202_GBL9007178.txt']
ERK_module_output = ['MAPK_pT202_GBL9007178.txt']
ERK_module_output_diag = ['MAPK_pT202_GBL9007178.txt']

AKT_module_names = ['p85PI3K_GBL9006917.txt','PI3K_p110a.C._GBL9006918.txt','AKT_pS473_V_GBL9006995.txt','AKT_pT308_V_GBL9006996.txt','AKT_V_GBL9006994.txt','AKT_pS473_V_GBL9006969.txt','PDK1_pS241_V_GBL9007265.txt','PDK1_V_GBL9006961.txt','PDK1_pS241_V_GBL9007273.txt','PDK1.V._GBL9006984.txt']
AKT_module_output = ['AKT_pS473_V_GBL9006995.txt']
AKT_module_output_diag = ['AKT_pS473_V_GBL9006995.txt']

TOR_module_names = ['mTOR_V_GBL9007482.txt','mTOR_pS2448_GBL9007262.txt','mTOR_pS2448_GBL9007124.txt','p70S6K_V_GBL9007296.txt','p70S6K_pT389_V_GBL9007115.txt','p70S6K_pT389.V._GBL9007103.txt']
TOR_module_output = ['p70S6K_pT389_V_GBL9007115.txt'] # pS6K
TOR_module_output_diag = ['p70S6K_pT389_V_GBL9007115.txt'] # same

# maybe for SRC we should pick other diagnal element
SRC_module_names = ['SRC_V_mouse_GBL9007307.txt','SRC_pY416_C_GBL9007263.txt','SRC_pY527_V_GBL9007268.txt','SRC_GBL9007055.txt','SRC_pY416_C_GBL9007276.txt','SRC_pY527_V_GBL9007272.txt','SRC_GBL9007059.txt','SRC_pY317_GBL9007060.txt']
SRC_module_output = ['SRC_pY416_C_GBL9007263.txt']
SRC_module_output_diag = ['SRC_pY416_C_GBL9007263.txt']

CDK_module_names = ['CyclinD1_V_GBL9007009.txt','Cyclin.D1_V_GBL9007283.txt','Rb_V_mouse_GBL9007306.txt','Rb_pS807_V_GBL9007269.txt','Rb_pS807_V_GBL9007274.txt','Rb_pS807_V_GBL9007274.txt'] #
CDK_module_output = ['Rb_pS807_V_GBL9007274.txt']
CDK_module_output_diag = ['Rb_pS807_V_GBL9007274.txt']

PKC_module_names = ['PKCa_V_mouse_GBL9007305.txt','PKCa_pS657_V_GBL9007267.txt','PKCdelta_pS664_GBL9007266.txt','PKCa_pS657_V_GBL9007127.txt','PKCdelta_pS664_GBL9007126.txt','PKCdelta_pS664_GBL9006912.txt']
PKC_module_output = ['PKCa_pS657_V_GBL9007267.txt'] # phospho-PKCa
PKC_module_output_diag = ['p70S6K_pT389_V_GBL9007115.txt'] # phospho-pS6K

IRS_module_names = ['IGFBP2_V_GBL9006949.txt','IGFRb_C_GBL9006950.txt','IRS1_V_GBL9006951.txt','IRS1_pS307_GBL9007260.txt','IRS1_pS307_GBL9007275.txt','IRS1_pS307_GBL9007066.txt'] #
IRS_module_output = ['IRS1_V_GBL9006951.txt','IRS1_pS307_GBL9007066.txt'] # total IRS - phospho-IRS
IRS_module_output_diag = ['IRS1_V_GBL9006951.txt','IRS1_pS307_GBL9007066.txt'] # none actually


total_proteins = []

#treatments_output = ["JNKinh","MEKinh","AKTinh","JNK8inh","S6Kinh","RSKinh"]
modules_output = [ERK_module_output,AKT_module_output,TOR_module_output,SRC_module_output,CDK_module_output,PKC_module_output,IRS_module_output]
modules_output_diag = [ERK_module_output_diag,AKT_module_output_diag,TOR_module_output_diag,SRC_module_output_diag,CDK_module_output_diag,PKC_module_output_diag,IRS_module_output_diag]

total_names = ERK_module_names + AKT_module_names + TOR_module_names + SRC_module_names + CDK_module_names + PKC_module_names + IRS_module_names + total_proteins
print(modules_output[6])

['IRS1_V_GBL9006951.txt', 'IRS1_pS307_GBL9007066.txt']


In [11]:
RPPA_total_red = RPPA_total.drop(columns=total_names)

drug12_ctrl_red = RPPA_total_red.loc[drugs12pert_ctrl.index]

drug12_AKTi_MEKi_red = RPPA_total_red.loc[drugs12pert_AKTi_MEKi.index]
drug12_PI3Ki_MEKi_red = RPPA_total_red.loc[drugs12pert_PI3Ki_MEKi.index]
drug12_mTORi_MEKi_red = RPPA_total_red.loc[drugs12pert_mTORi_MEKi.index]

drug12_dead_red = pd.concat([drug12_AKTi_MEKi_red,drug12_PI3Ki_MEKi_red,drug12_mTORi_MEKi_red],axis=0)
ctrl_cnt_red = drug12_ctrl_red.mean(axis=0)
dead_cnt_red = drug12_dead_red.mean(axis=0)

In [12]:
#building new STV

drug12_trainset_red = pd.concat([drug12_ctrl_red,drug12_AKTi_MEKi_red,drug12_PI3Ki_MEKi_red,drug12_mTORi_MEKi_red],axis=0)

svc_red = SVC(kernel='linear')
svc_red.fit(drug12_trainset_red,cls)
b_red = svc_red.intercept_[0]/linalg.norm(svc_red.coef_[0])
n_vec_red = - svc_red.coef_[0]/linalg.norm(svc_red.coef_[0])
s_vec_red = (ctrl_cnt_red - dead_cnt_red)/linalg.norm(ctrl_cnt_red - dead_cnt_red)

norm_vec_df_red = pd.DataFrame(s_vec_red,index=RPPA_total_red.columns,columns=['s_vec_red'])
norm_vec_df_red.insert(0,'norm_vec_red',n_vec_red)
# plot STV ranking
norm_vec_df_red.to_csv('Sander_reduced_STV.csv')
norm_vec_df_red.sort_values(by='s_vec_red')

# minus is higher in VEM-sensitive state
# plus is higher in VEM-resistant state

,norm_vec_red,s_vec_red
Caveolin_V_GBL9007006.txt,-0.154731,-0.114581
Calveolin.V._GBL9007280.txt,-0.131328,-0.104275
b.Catenin_V_GBL9007000.txt,-0.231173,-0.102881
b.Catenin.V._GBL9007278.txt,-0.153899,-0.053474
YAP_V_GBL9006957.txt,-0.031596,-0.041715
...,...,...
S6_pS240.V._GBL9007070.txt,0.210056,0.172661
S6_pS235_V_GBL9007270.txt,0.319261,0.271241
CyclinB1.V._GBL9007281.txt,0.169088,0.292470
CyclinB1_V_GBL9007008.txt,0.218753,0.324324


In [13]:
angle_red = (np.arccos(np.dot(n_vec_red,s_vec_red))/np.pi)*180
angle_red

23.25101636863223

In [14]:
norm_vec_df_red.filter(regex="Myc",axis=0)
# Myc has positive contribution to STV, OK

,norm_vec_red,s_vec_red
c.Myc_GBL9006959.txt,0.005071,0.002703
c.Myc_pT58_GBL9007257.txt,-0.003729,-0.000747
c.Myc_pT58_GBL9007121.txt,0.000904,-0.000966
c.Myc_pT58_GBL9007179.txt,-0.000745,-0.000751


In [15]:
# calculate DPDs

DPDs = pd.DataFrame(columns=['DPD_value'],index=drugs12pert.index)

for smpl in DPDs.index:
    DPD_val = -(b_red - np.dot(RPPA_total_red.loc[smpl],norm_vec_df_red['norm_vec_red']))/np.dot(norm_vec_df_red['s_vec_red'],norm_vec_df_red['norm_vec_red'])
    DPDs.at[smpl,'DPD_value'] = DPD_val
DPDs.insert(1,'Sample Description (drug abbre. | dose or time-point)',drugs12pert['Sample Description (drug abbre. | dose or time-point)'])
DPDs.insert(2,'series / replications  on one slide',drugs12pert['series / replications  on one slide'])

DPDs.loc[drugs12pert_ctrl.index].sort_values(by='Sample Description (drug abbre. | dose or time-point)')

,DPD_value,Sample Description (drug abbre. | dose or time-point),series / replications on one slide
Sample Name (in RPPA data from Yiling),,,
wq|38,7.10693,ctrl 1,r1
wq|1,9.62971,ctrl 2,r1
wq|82,9.94127,ctrl 3,r1
wq|149,13.0078,ctrl 4,r2
wq|218,12.213,ctrl 5,r3
wq|265,9.28036,ctrl 6,r3
wq|171,8.65455,ctrl 7,r2


In [16]:
DPDs.loc[drugs12pert_mTORi.index].sort_values(by='Sample Description (drug abbre. | dose or time-point)')

,DPD_value,Sample Description (drug abbre. | dose or time-point),series / replications on one slide
Sample Name (in RPPA data from Yiling),,,
wq|108,-6.97053,Tm|0.3,r2
wq|209,-6.85901,Tm|0.3,r3
wq|21,-6.26865,Tm|0.3,r1
wq|54,-7.94683,Tm|0.6,r1
wq|184,-8.63383,Tm|0.6,r2
wq|264,-9.08986,Tm|0.6,r3


In [17]:
# summary

# full dataset:
# the dataset itself (samples are in rows and analytes are in columns): drug12_trainset 
# labeling of the samples: cls
# SVM model: svc
# SVM model was created by the following command: svc.fit(drug12_trainset,cls)
# normal to the separation hyperplane (length = 1): norm_vec
# vector conneting centroids (length = 1): s_vec
# angle between these vectors: angle

# reduced dataset:
# the dataset itself (samples are in rows and analytes are in columns): drug12_ctrl_red
# labeling of the samples: cls
# SVM model: svc_red
# SVM model was created by the following command: svc_red.fit(drug12_trainset_red,cls)
# normal to the separation hyperplane (length = 1): n_vec_red
# vector conneting centroids (length = 1): s_vec_red
# angle between these vectors: angle_red


In [18]:
drug12_trainset.to_csv('drug12.csv')
drug12_trainset

,MEK1-2-pS217-221_GBL9006861.txt (correct),X4EBP1_pS65_V_GBL9007440.txt,X4EBP1_pT37_V_GBL9007441.txt,X4EBP1_pT70_GBL9007442.txt,X4EBP1_V_GBL9007439.txt,ACC1_C_GBL9006993.txt,AKT_pS473_V_GBL9006995.txt,AKT_pT308_V_GBL9006996.txt,AKT_V_GBL9006994.txt,AMPK_pT172_V_GBL9006997.txt,...,PKCdelta_pS664_GBL9006912.txt,PLK1_GBL9006923.txt,SMAC_DIABO.mouse._GBL9006859.txt,STAT5.V._GBL9006926.txt,TAZ.V._GBL9007105.txt,TAZ_pS89.C._GBL9007106.txt,Telomerase.C._GBL9007180.txt,XRCC1_GBL9007104.txt,YAP_pS127_GBL9007181.txt,YBI_pS102_GBL9006913.txt
Sample Name (in RPPA data from Yiling),,,,,,,,,,,,,,,,,,,,,
wq|38,1.243625,2.327408,4.107728,0.910308,0.547319,2.097788,0.966904,0.461380,4.066718,2.594595,...,0.619678,0.885327,1.224569,1.192498,1.434778,0.809492,0.781246,0.727818,1.831565,1.198131
wq|1,1.514585,3.017927,6.984166,1.264595,0.716835,1.963014,1.232042,0.477862,4.557528,2.548497,...,0.602309,0.739436,1.682960,1.163860,1.529380,0.771793,0.903152,0.719172,2.670085,0.950257
wq|82,1.515557,2.321149,4.169651,0.846003,0.480975,2.249646,1.180797,0.499094,3.539966,2.438710,...,0.660590,0.963642,0.638340,1.405922,1.841520,0.788579,0.787187,0.768586,1.646580,1.095284
wq|149,1.515472,1.879423,4.553797,0.946963,0.504934,2.144030,1.243053,0.480249,2.853227,1.950488,...,0.644691,1.060259,0.629050,1.422697,1.907430,0.842822,0.815716,0.837744,1.595816,1.104835
wq|171,1.173016,2.200609,3.921614,0.929788,0.447344,2.078094,1.287823,0.525600,3.907056,1.437128,...,0.700899,0.997850,0.638905,1.635724,1.696342,0.757311,0.848160,0.715345,1.611489,0.929147
wq|218,1.453980,2.357429,6.281565,0.952818,0.491019,1.988244,1.124459,0.499125,3.669221,2.582676,...,0.637425,0.911998,1.056908,1.443657,1.672488,0.737656,0.820306,0.660102,1.757141,1.027145
wq|265,1.515154,2.535593,5.258256,0.938048,0.473943,2.127090,1.297397,0.486526,3.202458,2.337713,...,0.655249,0.911409,0.914319,1.619381,1.607153,0.739238,0.694963,0.713492,1.545806,0.943545
wq|3,0.877327,1.765849,6.447800,0.984942,0.721063,2.148452,0.255605,0.142399,4.280753,2.075290,...,0.648980,0.598099,1.937342,1.261487,1.446399,0.777083,0.942528,0.651991,1.944894,0.616173
wq|126,0.885718,1.247748,3.574879,0.931688,0.506384,2.379339,0.249937,0.118326,2.909453,1.231543,...,0.683211,0.748340,1.094353,1.574567,1.821358,0.908740,0.836020,0.820292,1.609995,0.652330


In [19]:
drug12_ctrl_red.to_csv('drug12_red.csv')
drug12_ctrl_red

,X4EBP1_pS65_V_GBL9007440.txt,X4EBP1_pT37_V_GBL9007441.txt,X4EBP1_pT70_GBL9007442.txt,X4EBP1_V_GBL9007439.txt,ACC1_C_GBL9006993.txt,AMPK_pT172_V_GBL9006997.txt,AMPKa_C_GBL9006998.txt,AR_V_GBL9006999.txt,b.Catenin_pS33_C_GBL9007001.txt,b.Catenin_V_GBL9007000.txt,...,PCNA.mouse._GBL9006860.txt,PLK1_GBL9006923.txt,SMAC_DIABO.mouse._GBL9006859.txt,STAT5.V._GBL9006926.txt,TAZ.V._GBL9007105.txt,TAZ_pS89.C._GBL9007106.txt,Telomerase.C._GBL9007180.txt,XRCC1_GBL9007104.txt,YAP_pS127_GBL9007181.txt,YBI_pS102_GBL9006913.txt
Sample Name (in RPPA data from Yiling),,,,,,,,,,,,,,,,,,,,,
wq|38,2.327408,4.107728,0.910308,0.547319,2.097788,2.594595,0.778906,0.293340,0.626315,2.588177,...,0.775104,0.885327,1.224569,1.192498,1.434778,0.809492,0.781246,0.727818,1.831565,1.198131
wq|1,3.017927,6.984166,1.264595,0.716835,1.963014,2.548497,0.702767,0.265075,0.780027,5.563984,...,0.729919,0.739436,1.682960,1.163860,1.529380,0.771793,0.903152,0.719172,2.670085,0.950257
wq|82,2.321149,4.169651,0.846003,0.480975,2.249646,2.438710,0.821289,0.270695,0.855889,3.688558,...,0.784792,0.963642,0.638340,1.405922,1.841520,0.788579,0.787187,0.768586,1.646580,1.095284
wq|149,1.879423,4.553797,0.946963,0.504934,2.144030,1.950488,0.832651,0.269405,1.056437,5.108547,...,0.837015,1.060259,0.629050,1.422697,1.907430,0.842822,0.815716,0.837744,1.595816,1.104835
wq|171,2.200609,3.921614,0.929788,0.447344,2.078094,1.437128,0.796014,0.308694,0.917606,3.878137,...,0.720518,0.997850,0.638905,1.635724,1.696342,0.757311,0.848160,0.715345,1.611489,0.929147
wq|218,2.357429,6.281565,0.952818,0.491019,1.988244,2.582676,0.817209,0.317491,0.971246,3.838676,...,0.713287,0.911998,1.056908,1.443657,1.672488,0.737656,0.820306,0.660102,1.757141,1.027145
wq|265,2.535593,5.258256,0.938048,0.473943,2.127090,2.337713,0.797780,0.286370,0.883317,3.610097,...,0.734758,0.911409,0.914319,1.619381,1.607153,0.739238,0.694963,0.713492,1.545806,0.943545
